# ММС. Лабораторная работа №2.

Исследуютя **детерминированные** стратегии в игре
"Дилемма заключенного"

* https://ru.wikipedia.org/wiki/%D0%94%D0%B8%D0%BB%D0%B5%D0%BC%D0%BC%D0%B0_%D0%B7%D0%B0%D0%BA%D0%BB%D1%8E%D1%87%D1%91%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE
* https://www.youtube.com/watch?v=o-lFK7nZIKE
## Описание игры
Соперничают 2 игрока: $A$ и $B$. Каждую партию стороны одновременно выбирают: сотрудничать ($0$) или предать ($1$).
Таблица выиграша $W$ имеет вид:

|         | 0   | 1   |
| ------- | --- | --- |
| **0**   | 3   | 0   |
| **1**   | 5   | 1   |

Пусть $A$ выбрал ход $i$, а $B$ ход $j$. 
Тогда:
* выигрыш A есть $W[i][j]$
* выигрыш B есть $W[j][i]$

Игра состоит из $200$ партий.
Цель каждого игрока - получить максимальное число очков к концу игры.

## Задача
Необходимо реализовать и сравнить между собой некоторые **детерменированные** стратегии.

**Стратегия** - алгоритм/принцип, по которому на каждом ходе игры принимается решение (0 или 1), основываясь на исходах предыдущих партий.

Результаты сравнения стратегий по каждой метрике необходимо представить в виде отдельной таблицы.

## Метрики
Стратегии сравниваются между собой по следующим показателям:
1. Общее число очков
2. Длина наибольшей доминирующей серии (когда один получает +5, а другой 0 очков)

## Стратегии
- Alex - всегда 1
- Bob - всегда 0
- Clara - сначала 0, потом последнее значение опонента
- Denis - сначала 0, потом обратное к последнему выбору опонента
- Emma - всегда 0, но каждая 20 партия - 1
- Frida - 0 до тех пор, пока у опонента 0. затем 1
- George - Придумайте свою детерминированную стратегию

In [89]:
#do not trust always
def AlexMove(movesMy, movesOp):
    move = 1
    return move
# trust always
def BobMove(movesMy, movesOp):
    move = 0
    return move
#begin with trust, then copy op move
def ClaraMove(movesMy, movesOp):
    if not movesOp:
        move = 0
    else:
        move = movesOp[-1]
    return move
    
#begin with trust, then do oposite op move
def DenisMove(movesMy, movesOp):
    if not movesOp:
        move = 0
    else:
        move = 0 if (movesOp[-1]) else 1
    return move


def EmmaMove(movesMy, movesOp):
    return (1 if (len(movesMy)+1) % 20== 0 else 0)
    
def FridaMove(movesMy, movesOp):        
    return 1 if sum(movesOp)>0 else 0

def GeorgeMove(movesMy, movesOp):
    gap = 3
    trust_threshold = 5
    limit = 4
    #trusts for some time
    if len(movesMy) < trust_threshold:
        return 0
    # trying to fool
    if len(movesMy) == trust_threshold:
        return 1

    # than trusts again for some time
    if len(movesMy) <= trust_threshold + gap:
        return 0

    #if not punished, fools, if punished proceeds with respect, if was fooled to many times fooling till end of game
    if sum(movesOp[trust_threshold:trust_threshold + gap]) == 0:
        return 1
    elif sum(movesOp[:trust_threshold + gap]) > limit:
        return 1
    else:
        return movesOp[-1]


In [90]:
import numpy as np

def PlayGame(player1, player2, show = False):
    WinTable = [ [3, 0], [ 5, 1]]
    movesFirst = []
    movesSecond = []

    
    Scores_p1 = []   
    for x in range(200):
        move1 = int(player1(movesFirst, movesSecond))
        move2 = int(player2(movesSecond, movesFirst))
        movesFirst.append(move1)
        movesSecond.append(move2)
        if (show):
            print("game #", x)
            print("Trust" if not move1 else "Fool", "Trust" if not move2 else "Fool")
            print("+", WinTable[move1][move2], "   +", WinTable[move2][move1], "\n")
            
        Scores_p1.append(WinTable[move1][move2])
        #Score_p2 += WinTable[move2][move1]    
    return (Scores_p1)




In [91]:
Strat = {"Alex" : AlexMove, "Bob" :BobMove, "Clara": ClaraMove, "Denis" : DenisMove, "Emma" : EmmaMove, "Frida" : FridaMove, "George" : GeorgeMove } 
Players = ["Alex", "Bob", "Clara", "Denis", "Emma", "Frida", "George"]

In [92]:
import pandas as pd

matr = [[]]
for p1 in Players:
    line = []
    for p2 in Players:
        line.append(PlayGame(Strat[p1], Strat[p2]))
    matr.append(line)
matr.pop(0) 

GamesData = pd.DataFrame(matr, index=Players, columns=Players)     


print("line = player\ncolumn = opponent\nitem = how much player gets")


ScoresData = GamesData.map(sum)
print(ScoresData)
print("--------------------------------------------------------\n    sum of scores against all strategies")

ScoresDataSum = ScoresData.sum(axis=1)

print(ScoresDataSum)

#first argumetn is opponent, second if player
print(GamesData["Bob"]["Emma"])
print(GamesData["Clara"]["George"])

line = player
column = opponent
item = how much player gets
        Alex   Bob  Clara  Denis  Emma  Frida  George
Alex     200  1000    204   1000   960    204     232
Bob        0   600    600      3   570    600      24
Clara    199   600    600    450   588    600     599
Denis      0   998    450    400   940      9      38
Emma      10   620    593     40   580     71      34
Frida    199   600    600    989   921    600     221
George   192   984    599    958   944    211     598
--------------------------------------------------------
    sum of scores against all strategies
Alex      3800
Bob       2397
Clara     3636
Denis     2835
Emma      1948
Frida     4130
George    4486
dtype: int64
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

Предавать выгоднее всего, Алекс собирает больше всего очков против каждой из стратегий.

In [93]:
def CountSeries(arr):
    maxSeries = 0
    counter = 0
    for item in arr:
        if item == 5:
            counter+=1
        if counter > maxSeries:
            maxSeries = counter
    return maxSeries
    

### Исследование на максимальную длину доминирующей серии

In [94]:
SeriesData = GamesData.map(CountSeries)
print(SeriesData)
print("--------------------------------------------------------\n    sum of scores against all strategies (by series length)")
print( SeriesData.sum(axis=1))

        Alex  Bob  Clara  Denis  Emma  Frida  George
Alex       0  200      1    200   190      1       8
Bob        0    0      0      0     0      0       0
Clara      0    0      0     50     9      0       1
Denis      0  199     50      0   180      1       6
Emma       0   10     10      0     0      1       0
Frida      0    0      0    197   171      0       3
George     0  192      1    190   182      1       0
--------------------------------------------------------
    sum of scores against all strategies (by series length)
Alex      600
Bob         0
Clara      60
Denis     436
Emma       21
Frida     371
George    566
dtype: int64
